In [ ]:
import cv2
import glob
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from flaskapp.imaging import MicroStructure

dir_path = "data/base microstructure"
img_paths = sorted(glob.glob("{}/image_*.png".format(dir_path)))
gb_mask_paths = sorted(glob.glob("{}/gbmask_*.png".format(dir_path)))
lam_mask_paths = sorted(glob.glob("{}/lammask_*.png".format(dir_path)))
p1_mask_paths = sorted(glob.glob("{}/p1mask_*.png".format(dir_path)))
p2_mask_paths = sorted(glob.glob("{}/p2mask_*.png".format(dir_path)))

print(len(img_paths))
print(len(gb_mask_paths))
print(len(lam_mask_paths))
print(len(p1_mask_paths))
print(len(p2_mask_paths))

assert len(img_paths) == len(gb_mask_paths) == len(lam_mask_paths) == len(p1_mask_paths) == len(p2_mask_paths)

# plot_all([img, gb_mask, prec_mask, lam_mask, p1_mask, p2_mask])

In [ ]:
def visit(img_path, gb_mask_path, lam_mask_path, p1_mask_path, p2_mask_path):
    mc = MicroStructure(img_path)
    
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    gb_mask = cv2.imread(gb_mask_path, cv2.IMREAD_GRAYSCALE)
    lam_mask = cv2.imread(lam_mask_path, cv2.IMREAD_GRAYSCALE)
    p1_mask = cv2.imread(p1_mask_path, cv2.IMREAD_GRAYSCALE)
    p2_mask = cv2.imread(p2_mask_path, cv2.IMREAD_GRAYSCALE)
    
    images = (
        gb_mask, 
        mc.grain_mask, 
        lam_mask, 
        mc.line_mask, 
        p1_mask, 
        mc.dark_mask, 
        p2_mask, 
        mc.white_mask
    )
    titles = ( 
        "correct grain boundary",
        "predicted grain boundary",
        "correct line phase",
        "predicted line phase",
        "correct dark phase",
        "predicted dark phase",
        "correct white phase",
        "predicted white phase",
    )
    
    columns = 2
    rows = 5
    fig, axs = plt.subplots(rows, columns, figsize=(30,18*3-9), constrained_layout=True)
    fig.suptitle(img_path, fontsize=30)
    axs = axs.flatten()
    
    axs[0].imshow(img, cmap="gray")
    axs[0].set_title("original")
    axs[0].axis('off')
    
    def compute_phase_fraction(mask):
        pixels = mask.flatten()
        white_pixels = len([p for p in pixels if p])
        fraction = white_pixels / len(pixels)
        return fraction
    
#     def compute_diameters_line_approx(image):
#         w = image.shape[0]
#         h = image.shape[1]
        
#         # mid horizontal approx
#         for i in range(w):
    
    correct_dark_frac = compute_phase_fraction(p1_mask)
    correct_light_frac = compute_phase_fraction(p2_mask)
    correct_line_frac = compute_phase_fraction(lam_mask)
    
    table_data=[
        ["metric", "correct", "predicted", "error"],
        ["dark frac", "%.3f"%correct_dark_frac, "%.3f"%mc.dark_fraction, "{0:.3f}%".format(abs(correct_dark_frac-mc.dark_fraction)*100)],
        ["white frac", "%.3f"%correct_light_frac, "%.3f"%mc.light_fraction, "{0:.3f}%".format(abs(correct_light_frac-mc.light_fraction)*100)],
        ["line frac", "%.3f"%correct_line_frac, "%.3f"%mc.line_fraction, "{0:.3f}%".format(abs(correct_line_frac-mc.line_fraction)*100)],
        ["ave grain diameter", "n/a", "%.3f"%mc.average_grain_diameter, "n/a"],
    ]
    table = axs[1].table(cellText=table_data, loc='best')
    table.set_fontsize(16)
    table.scale(1,3)
    axs[1].set_title("metrics")
    axs[1].axis('off')
    
    for ax, image, title in zip(axs[2:], images, titles):
        ax.imshow(image, cmap="gray")
        ax.set_title(title)
        ax.axis('off')
    
    return fig

In [ ]:
pp = PdfPages('foo.pdf')

total = len(img_paths)
for i,pths in enumerate(zip(img_paths, gb_mask_paths, lam_mask_paths, p1_mask_paths, p2_mask_paths)):
    if i % 10 == 0:
        print("{}/{}".format(i, total))
    pp.savefig(visit(*pths))

pp.close()